<a href="https://colab.research.google.com/github/jydiw/nyt-covid-19-data/blob/master/eda_nytimes-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combining NYTimes data with county census and geoson data

In [1]:
import numpy as np
import numpy.polynomial.polynomial as poly      # linear regression
import pandas as pd

from scipy.signal import savgol_filter          # fast smoothing of data

# opening external coordinates
import json
import pickle

# opening urls
from urllib.request import urlopen

# benchmarking
from time import time

# date/time operations
from datetime import datetime, timedelta
from pytz import timezone

import json

# plotting
import matplotlib.pyplot as plt

# import nytimes data

New York Times data has a few caveats, including treating New York City, Kansas City, and Joplin as single entities rather than including them in their respective counties. Read their [README](https://github.com/nytimes/covid-19-data/blob/master/README.md) for more information.

In [2]:
def optimize(df):
    '''
    Optimizes the data types in a pandas dataframe.
    '''
    dft = df.copy()
    # converts to datetime if possible
    dft = dft.apply(lambda col:
        pd.to_datetime(col, errors='ignore') if col.dtypes=='object' else col)
    
    # if there are less than half as many unique values as there are rows, convert to category
    for col in dft.select_dtypes(include='object'):
        if len(dft[col].unique()) / len(df[col]) < 0.5:
            dft[col] = dft[col].astype('category')
            
    # downcasts numeric columns if possible
    dft = dft.apply(lambda col: 
        pd.to_numeric(col, downcast='integer') if col.dtypes=='int64' else col)
    dft = dft.apply(lambda col: 
        pd.to_numeric(col, downcast='float') if col.dtypes=='float64' else col)
    
    return dft

In [4]:
# with urlopen('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv') as response:
#     usaf_cases = optimize(pd.read_csv(response, dtype={'countyFIPS':'str', 'stateFIPS':'str'}))
# usaf_cases.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,2010,2021,2023,2030,2048,2059,2074,2082,2103,2126
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,6405,6443,6475,6615,6637,6658,6694,6712,6743,6768
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,988,996,997,1012,1031,1033,1033,1042,1045,1055
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,791,801,811,825,828,840,843,850,856,861


In [6]:
usaf_cases[usaf_cases['County Name'] == 'Jasper County']

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20
477,13159,Jasper County,GA,13,0,0,0,0,0,0,...,241,241,241,243,243,244,246,246,247,249
649,17079,Jasper County,IL,17,0,0,0,0,0,0,...,250,257,258,260,261,264,264,266,272,275
749,18073,Jasper County,IN,18,0,0,0,0,0,0,...,564,576,596,619,645,667,690,709,725,741
855,19099,Jasper County,IA,19,0,0,0,0,0,0,...,1020,1031,1041,1064,1072,1090,1102,1116,1131,1174
1457,28061,Jasper County,MS,28,0,0,0,0,0,0,...,674,677,679,687,687,691,692,696,698,702
1558,29097,Jasper County,MO,29,0,0,0,0,0,0,...,4893,4955,5028,5091,5159,5170,5235,5329,5329,5452
2385,45053,Jasper County,SC,45,0,0,0,0,0,0,...,883,885,895,898,906,915,916,919,923,929
2689,48241,Jasper County,TX,48,0,0,0,0,0,0,...,509,511,512,512,512,512,515,515,515,515


In [4]:
with urlopen('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv') as response:
    csse_cases = optimize(pd.read_csv(response, dtype={'FIPS':'str'}))
csse_cases.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539528,-86.644081,...,1966,1983,1989,1999,2010,2021,2023,2030,2048,2059
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727751,-87.722069,...,6333,6350,6369,6375,6405,6443,6475,6615,6637,6658
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387131,...,968,977,981,981,988,996,997,1012,1031,1033
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996422,-87.125114,...,771,775,785,789,791,801,811,825,828,840
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567909,...,1783,1807,1827,1838,1848,1873,1893,1911,1925,1932


In [3]:
with urlopen('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv') as response:
    nyt_df_raw = optimize(pd.read_csv(response, dtype={'fips':'str'}))
nyt_df_raw.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


In [8]:
nyt_df_raw[nyt_df_raw['fips'] == '29097']

,date,county,state,fips,cases,deaths
6629,2020-03-21,Jasper,Missouri,29097,1,0
7718,2020-03-22,Jasper,Missouri,29097,1,0
8906,2020-03-23,Jasper,Missouri,29097,1,0
10206,2020-03-24,Jasper,Missouri,29097,1,0
11628,2020-03-25,Jasper,Missouri,29097,2,0
...,...,...,...,...,...,...
665692,2020-10-25,Jasper,Missouri,29097,3198,41
668937,2020-10-26,Jasper,Missouri,29097,3201,41
672179,2020-10-27,Jasper,Missouri,29097,3242,43
675422,2020-10-28,Jasper,Missouri,29097,3276,44


In [4]:
nyt_df_raw['fips'] = nyt_df_raw['fips'].astype('object')
nyt_df_raw.loc[nyt_df_raw['county'] == 'New York City','fips'] = '36NYC'
nyt_df_raw.loc[nyt_df_raw['county'] == 'Kansas City','fips'] = '29KCM'
nyt_df_raw.loc[nyt_df_raw['county'] == 'Joplin','fips'] = '29JOP'
nyt_df_raw['fips'] = nyt_df_raw['fips'].astype('category')

In [5]:
nyt_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693342 entries, 0 to 693341
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    693342 non-null  datetime64[ns]
 1   county  693342 non-null  category      
 2   state   693342 non-null  category      
 3   fips    687315 non-null  category      
 4   cases   693342 non-null  int32         
 5   deaths  693342 non-null  int16         
dtypes: category(3), datetime64[ns](1), int16(1), int32(1)
memory usage: 12.8 MB


# import `info_df`

In [6]:
with open('../data/processed/info_df.p', 'rb') as f:
    info_df = pickle.load(f)
print(info_df.shape)
info_df.head()

(3140, 231)


,state_fips,state,county,fips,tot_pop,tot_male,tot_female,tot_pop_white_male,tot_pop_white_female,tot_pop_black_male,...,per_pop_hispanic_male,per_pop_hispanic_female,per_pop_white,per_pop_black,per_pop_native,per_pop_asian,per_pop_pacific,per_pop_twoplus,per_pop_hispanic,per_votes
0,01,Alabama,Autauga,01001,55869,27092.0,28777.0,20138.0,21077.0,5171.0,...,0.015823,0.014087,0.737708,0.198643,0.004349,0.011563,0.000716,0.017111,0.029909,0.441408
1,01,Alabama,Baldwin,01003,223234,108247.0,114987.0,89845.0,95902.0,9308.0,...,0.024839,0.022349,0.832073,0.086076,0.006751,0.010509,0.000551,0.016852,0.047188,0.421486
2,01,Alabama,Barbour,01005,24686,13064.0,11622.0,5894.0,5341.0,6260.0,...,0.025480,0.019768,0.455116,0.478287,0.003848,0.004699,0.001256,0.011545,0.045248,0.420886
3,01,Alabama,Bibb,01007,22394,11929.0,10465.0,8482.0,8181.0,2912.0,...,0.015317,0.012503,0.744083,0.210726,0.004064,0.002054,0.000268,0.010985,0.027820,0.390640
4,01,Alabama,Blount,01009,57826,28472.0,29354.0,24494.0,25682.0,453.0,...,0.051015,0.045516,0.867707,0.015080,0.004877,0.002819,0.000363,0.012624,0.096531,0.438972


In [7]:
with open('../data/processed/geo_json.json') as f:
    geo_json = json.load(f)

## engineer per capita columns

In [8]:
nyt_df = nyt_df_raw.merge(
    info_df[['fips', 'tot_pop']], 
    on='fips', 
    suffixes=('_x','')
)

# df_all = df_all.drop(['county_x', 'state_x'], axis=1)
nyt_df[['cases_per_100k', 'deaths_per_100k']] = nyt_df[['cases', 'deaths']].div(nyt_df['tot_pop'], axis=0) * 100_000
nyt_df = nyt_df.drop(columns=['tot_pop'])
nyt_df = nyt_df.sort_values(by=['date', 'fips'])

print(nyt_df.shape)
nyt_df.head()

(625061, 8)


,date,county,state,fips,cases,deaths,cases_per_100k,deaths_per_100k
0,2020-01-21,Snohomish,Washington,53061,1,0,0.121642,0.0
1,2020-01-22,Snohomish,Washington,53061,1,0,0.121642,0.0
2,2020-01-23,Snohomish,Washington,53061,1,0,0.121642,0.0
272,2020-01-24,Cook,Illinois,17031,1,0,0.019417,0.0
3,2020-01-24,Snohomish,Washington,53061,1,0,0.121642,0.0


## engineer change columns, window sum, and smooth

In [7]:
def add_change_cols(df, cols, pre='new_', clip=False):
    df = df.sort_values(by=['date', 'fips'])
    new_cols = [pre + c for c in cols]
    df[new_cols] = df[cols] - df.groupby(by='fips')[cols].shift()
    df[new_cols] = df[new_cols].fillna(0)
    df[new_cols] = df[new_cols].astype(int)
    if clip:
        df[new_cols] = df[new_cols].clip(lower=0)
    return (df, new_cols)

def add_window_cols(df, cols, window=7):
    df = df.sort_values(by=['date', 'fips'])
    new_cols = [c + '_' + str(window) + 'd' for c in cols]
    col_dict = dict(zip(cols, new_cols))
    df = (df.merge(df.sort_values(by=['date', 'fips'])
            .groupby('fips')
            .rolling(window, on='date', min_periods=0)[cols].mean()
            .rename(columns=col_dict), on=['fips', 'date']))
#     df[new_cols] = df[new_cols].astype(int)
    return (df, new_cols)

def add_savgol_cols(df, cols, window=7, clip=False):
    def my_savgol(x, w):
        if len(x) >= w:
            return savgol_filter(x, w, 1)
        else:
            new_window = int(np.ceil(len(x) / 2) * 2 - 1)
            if new_window <= 1:
                return x
            else:
                return savgol_filter(x, new_window, 1)
    df = df.sort_values(by=['date', 'fips'])
    new_cols = [c + '_' + str(window) + 'sg' for c in cols]
    df[new_cols] = df.groupby(by='fips')[cols].transform(lambda x: my_savgol(x, window))
    if clip:
        df[new_cols] = df[new_cols].clip(lower=0)
    return (df, new_cols)


cols = ['cases', 'deaths']
nyt_df, new_cols = add_change_cols(nyt_df_raw, cols, pre='new_', clip=False)
nyt_df, window_cols = add_window_cols(nyt_df, new_cols, 15)
nyt_df, sg_cols = add_savgol_cols(nyt_df, new_cols, 15)
nyt_df['days'] = ((nyt_df['date'] - nyt_df['date'].max()) / np.timedelta64(1, 'D')).astype('int')

# def add_cols(df, cols, window=15, sg=True, delta=False):
#     df, new_cols = add_change_cols(df, cols, pre='new_', clip=True)
#     df, window_cols = add_window_cols(df, new_cols, window)
#     if sg:
#         df, sg_cols = add_savgol_cols(df, new_cols, window, clip=True)
#     if delta:
#         df, delta_new_cols = add_change_cols(df, new_cols, pre='delta_')
#         df, delta_window_cols = add_window_cols(df, delta_new_cols, window)
#         if sg:
#             df, delta_sg_cols = add_savgol_cols(df, delta_new_cols, window)
    
#     df['mortality_rate'] = df['deaths'] / nyt_df['cases']
#     df[f'mortality_rate_{window}d'] = df[f'new_deaths_{window}d'] / df[f'new_cases_{window}d']
#     df.loc[(df[f'new_deaths_{window}d'] ==0), f'mortality_rate_{window}d'] = 0

#     return df

In [8]:
# nyt_df = add_cols(nyt_df, cols, 15, sg=True)
nyt_df['new_cases'].describe()

count    687315.000000
mean         13.577661
std          77.321071
min       -2321.000000
25%           0.000000
50%           1.000000
75%           7.000000
max       14129.000000
Name: new_cases, dtype: float64

In [9]:
nyt_df[nyt_df['new_cases']==-2321]

,date,county,state,fips,cases,deaths,new_cases,new_deaths,new_cases_15d,new_deaths_15d,new_cases_15sg,new_deaths_15sg,days
489326,2020-09-02,Middlesex,Massachusetts,25017,25411,2070,-2321,-1,-76.933333,4.266667,-93.4,2.333333,-61


Incidentally, there is barely any time advantage to only processing new dates. Therefore, we'll keep the workflow to processing the entire dataframe.

In [11]:
# nyt_df['days'] = ((nyt_df['date'] - nyt_df['date'].max()) / np.timedelta64(1, 'D')).astype('int')
# nyt_df['mortality_rate'] = nyt_df['deaths'] / nyt_df['cases']
# nyt_df['mortality_rate_15d'] = nyt_df['new_deaths_15d'] / nyt_df['new_cases_15d']
# # sometimes there are more deaths than new cases, but that doesn't mean more than 100% are dying
# nyt_df.loc[(nyt_df['mortality_rate_15d'] > 1), 'mortality_rate_15d'] = 1
# nyt_df['mortality_rate_15d'].describe()

count    575406.000000
mean          0.031441
std           0.089253
min           0.000000
25%           0.000000
50%           0.000000
75%           0.028571
max           1.000000
Name: mortality_rate_15d, dtype: float64

In [11]:
with open('../data/processed/nyt_df.p', 'wb') as file:
    pickle.dump(nyt_df, file, protocol=pickle.HIGHEST_PROTOCOL)
nyt_df.to_csv('../data/processed/nyt_df.csv', index=False)

In [12]:
nyt_df.columns

Index(['date', 'county', 'state', 'fips', 'cases', 'deaths', 'new_cases',
       'new_deaths', 'new_cases_15d', 'new_deaths_15d', 'new_cases_15sg',
       'new_deaths_15sg', 'days'],
      dtype='object')

In [20]:
nyt_df_w = nyt_df[nyt_df['days'] % 7 == 0]
nyt_df_w.shape

(78481, 19)

In [21]:
with open('../data/processed/nyt_df_w.p', 'wb') as file:
    pickle.dump(nyt_df_w, file, protocol=pickle.HIGHEST_PROTOCOL)
nyt_df_w.to_csv('../data/processed/nyt_df_w.csv', index=False)

# making clustered `nyt_df_ac`

In [ ]:
# import os
# import sys
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)

In [ ]:
with open('../data/processed/geo_json_ac.json') as f:
    geo_json_ac = json.load(f)

In [ ]:
dem_df_ac = optimize(pd.read_csv('../data/processed/dem_df_ac.csv', 
                                 dtype={'fips':'str', 'cluster':'int'}))
print(dem_df_ac.shape)
dem_df_ac.head()

In [ ]:
dem_df_ac.sort_values(by='total_pop').iloc[[0, -1]]

In [ ]:
dem_df_ac.sort_values(by='pop_density').iloc[[0, -1]]

There are several things to consider:

- Raw numbers will show New York City and Los Angeles County with the highest numbers, all other factors being equal.
- Reporting cases as a percentage (or as we'll use later, a per 100k) would better reflect the severity of infections.
- However, a higher population density would likely make it easier to transmit the disease to another person. Looking at the day-to-day change in new cases could allow us to see how quickly the disease is spreading.

The NYTimes dataset deals with New York City, Kansas City, and Joplin as their own entities. As such, they do not have valid `fips` codes; we must add our custom `fips` from our earlier census EDA.

In [ ]:
nyt_df['fips'] = nyt_df['fips'].astype('object')
nyt_df.loc[nyt_df['county'] == 'New York City','fips'] = '36NYC'
nyt_df.loc[nyt_df['county'] == 'Kansas City','fips'] = '29KCM'
nyt_df.loc[nyt_df['county'] == 'Joplin','fips'] = '29JOP'
nyt_df['fips'] = nyt_df['fips'].astype('category')

In [ ]:
nyt_df[nyt_df['fips']=='36NYC']

# merge `nyt_df` and `pop_df` for feature engineering

In [ ]:
dem_df_to_merge = optimize(pd.read_csv('../data/processed/dem_df_to_merge.csv', 
                                 dtype={'fips':'str', 'cluster':'int'}))
print(dem_df_to_merge.shape)
dem_df_to_merge.head()

To reduce the size of the resulting `csv`, we will only merge with the `tot_pop` column to engineer our per capita columns. We can merge with `pop_df` again when plotting our data.

In [ ]:
df = nyt_df.merge(
    dem_df_to_merge[['fips', 'cluster', 'state', 'county']], 
    on='fips', 
    suffixes=('_x','')
)

df = df.drop(['county_x', 'state_x'], axis=1)
df = df.sort_values(by=['date', 'fips'])
print(df.shape)
df.head()

In [ ]:
df_clustered = df.groupby(by=['state', 'cluster', 'date']).agg(
    cases=('cases', sum),
    deaths=('deaths', sum)
).dropna().reset_index().astype({'cases': 'int', 'deaths':'int'})

df_clustered.head()

In [ ]:
df = df_clustered.merge(dem_df_ac, on=['state', 'cluster'], suffixes=('_x',''))
df[['cases_per_100k', 'deaths_per_100k']] = df[['cases', 'deaths']].div(df['total_pop'], axis=0) * 100_000
df = df.sort_values(by=['date', 'fips'])

print(df.shape)
df.head()

In [ ]:
cols = ['cases', 'deaths', 'cases_per_100k', 'deaths_per_100k']

def add_change_cols(df, cols, pre='new_', clip=False):
    df = df.sort_values(by=['date', 'fips'])
    new_cols = [pre + c for c in cols]
    df[new_cols] = df[cols] - df.groupby(by='fips')[cols].shift()
    df[new_cols] = df[new_cols].fillna(0)
    if clip:
        df[new_cols] = df[new_cols].clip(lower=0)
    return (df, new_cols)

def add_savgol_cols(df, cols, window=7, clip=False):
    def my_savgol(x, w):
        if len(x) >= w:
            return savgol_filter(x, w, 1)
        else:
            new_window = int(np.ceil(len(x) / 2) * 2 - 1)
            if new_window <= 1:
                return x
            else:
                return savgol_filter(x, new_window, 1)
    df = df.sort_values(by=['date', 'fips'])
    cols_d = [c + '_' + str(window) + 'sg' for c in cols]
    df[cols_d] = df.groupby(by='fips')[cols].transform(lambda x: my_savgol(x, window))
    if clip:
        df[cols_d] = df[cols_d].clip(lower=0)
    return (df, cols_d)

df, new_cols = add_change_cols(df, cols, pre='new_', clip=True)
df, new_cols_7sg = add_savgol_cols(df, new_cols, clip=True)
df, new_cols_15sg = add_savgol_cols(df, new_cols, window=15, clip=True)
df, delta_new_cols = add_change_cols(df, new_cols, pre='delta_')
df, delta_new_cols_7sg = add_savgol_cols(df, delta_new_cols)
df, delta_new_cols_15sg = add_savgol_cols(df, delta_new_cols, window=15)

print(df.columns)
df.head()

In [ ]:
county = 'Los Angeles'
plt.plot(np.arange(len(df[df['county'] == county])), 'new_cases_per_100k', data=df[df['county'] == county], color='gray')
plt.plot(np.arange(len(df[df['county'] == county])), 'new_cases_per_100k_7sg', data=df[df['county'] == county], color='red')
plt.plot(np.arange(len(df[df['county'] == county])), 'new_cases_per_100k_15sg', data=df[df['county'] == county], color='blue')

In [ ]:
county = 'New York City'
plt.plot(np.arange(len(df[df['county'] == county])), 'new_cases_per_100k', data=df[df['county'] == county], color='gray')
plt.plot(np.arange(len(df[df['county'] == county])), 'new_cases_per_100k_7sg', data=df[df['county'] == county], color='red')
plt.plot(np.arange(len(df[df['county'] == county])), 'new_cases_per_100k_15sg', data=df[df['county'] == county], color='blue')